In [2]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input


In [3]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [4]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [5]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [6]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [7]:
model_resnet = Sequential()
model_resnet.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model_resnet.add(Dense(num_classes, activation='softmax'))
model_resnet.layers
model_resnet.layers[0].trainable = False
model_resnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [8]:
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2
fit_history_resnet = model_resnet.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 131s 417ms/step - loss: 0.0289 - accuracy: 0.9886 - val_loss: 0.0065 - val_accuracy: 0.9984
Epoch 2/2
301/301 [==============================] - 123s 410ms/step - loss: 0.0055 - accuracy: 0.9989 - val_loss: 0.0047 - val_accuracy: 0.9988


In [10]:
model_resnet.save('classifier_resnet_model.h5')

In [16]:
evaluate_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical',shuffle=False)

loss, accuracy = model_resnet.evaluate(evaluate_generator)
print('Model Resnet50: Loss:', loss, 'Accuracy:', accuracy)

Found 500 images belonging to 2 classes.
5/5 [==============================] - 2s 316ms/step - loss: 0.0024 - accuracy: 1.0000
Model Resnet50: Loss: 0.0024137087166309357 Accuracy: 1.0
